<a href="https://colab.research.google.com/github/juliaNogueiraC/Convolutional-Neural-Network-CNN-for-Image-Classification/blob/main/CNN_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar Dataset

In [ ]:
import os
import shutil
import random

# Defina os caminhos
download_path = 'caminho/para/pasta/train'  # Pasta onde você descompactou train.zip
dataset_path = 'dataset'  # Pasta onde você quer criar a estrutura de diretórios

# Crie a estrutura de diretórios
for folder in ['train/dogs', 'train/cats', 'validation/dogs', 'validation/cats']:
    os.makedirs(os.path.join(dataset_path, folder), exist_ok=True)

# Liste todos os arquivos
all_images = os.listdir(download_path)

# Separe cães e gatos
dog_images = [img for img in all_images if img.startswith('dog')]
cat_images = [img for img in all_images if img.startswith('cat')]

# Defina a proporção de validação (por exemplo, 20% para validação)
validation_split = 0.2

# Função para mover imagens
def move_images(images, animal_type):
    random.shuffle(images)
    split_index = int(len(images) * (1 - validation_split))

    for img in images[:split_index]:
        shutil.copy(os.path.join(download_path, img),
                    os.path.join(dataset_path, f'train/{animal_type}s', img))

    for img in images[split_index:]:
        shutil.copy(os.path.join(download_path, img),
                    os.path.join(dataset_path, f'validation/{animal_type}s', img))

# Mova as imagens
move_images(dog_images, 'dog')
move_images(cat_images, 'cat')

print("Dataset preparado com sucesso!")

# Rede neural convolucional (CNN) para classificação de imagens de cães e gatos.

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Definir parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 10
num_classes = 3

# Preparar os dados
train_data_dir = 'caminho/para/dados/treino'
validation_data_dir = 'caminho/para/dados/validacao'

# Aumentação de dados para treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Apenas reescala para validação
validation_datagen = ImageDataGenerator(rescale=1./255)

# Carregar e preparar os dados
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')

# Construir o modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Salvar o modelo
model.save('modelo_cnn_classificacao.h5')

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

## Version 1

exemplo com dataset baixado

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

# Definir parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 20  # Aumentado para 20 épocas
num_classes = 2  # Alterado para 2 classes: cães e gatos

# Preparar os dados
train_data_dir = 'dataset/train'
validation_data_dir = 'dataset/validation'

# Aumentação de dados para treinamento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Apenas reescala para validação
validation_datagen = ImageDataGenerator(rescale=1./255)

# Carregar e preparar os dados
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Alterado para 'binary' para classificação binária
    classes=['cats', 'dogs'])  # Especificar as classes explicitamente

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',  # Alterado para 'binary'
    classes=['cats', 'dogs'])  # Especificar as classes explicitamente

# Construir o modelo CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Alterado para 1 neurônio com ativação sigmoid
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Alterado para binary_crossentropy
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size)

# Salvar o modelo
model.save('modelo_cnn_caes_gatos.h5')

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

## Version 2 - Tensorflow

exemplo com dataset tensorflow

In [ ]:
!pip install tensorflow tensorflow-datasets

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Definir parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 20
num_classes = 2

# Carregar o dataset
(train_ds, validation_ds), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

# Função para pré-processar as imagens
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (img_height, img_width))
    return image, label

# Aplicar pré-processamento e criar batches
train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(batch_size)
validation_ds = validation_ds.map(preprocess_image).batch(batch_size)

# Aumentação de dados
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomZoom(0.1),
])

# Construir o modelo CNN
model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds
)

# Salvar o modelo
model.save('modelo_cnn_caes_gatos.h5')

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/23262 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.T6L2ZM_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


582/582 ━━━━━━━━━━━━━━━━━━━━ 853s 1s/step - accuracy: 0.6022 - loss: 0.6541 - val_accuracy: 0.7393 - val_loss: 0.5369
Epoch 2/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 816s 1s/step - accuracy: 0.7318 - loss: 0.5399 - val_accuracy: 0.7436 - val_loss: 0.5156
Epoch 3/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 824s 1s/step - accuracy: 0.7753 - loss: 0.4709 - val_accuracy: 0.8224 - val_loss: 0.3990
Epoch 4/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 841s 1s/step - accuracy: 0.8115 - loss: 0.4097 - val_accuracy: 0.8521 - val_loss: 0.3279
Epoch 5/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 868s 1s/step - accuracy: 0.8348 - loss: 0.3685 - val_accuracy: 0.8390 - val_loss: 0.3657
Epoch 6/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 863s 1s/step - accuracy: 0.8482 - loss: 0.3392 - val_accuracy: 0.8758 - val_loss: 0.2956
Epoch 7/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 861s 1s/step - accuracy: 0.8626 - loss: 0.3148 - val_accuracy: 0.8571 - val_loss: 0.3312
Epoch 8/20
582/582 ━━━━━━━━━━━━━━━━━━━━ 806s 1s/step - accuracy: 0.8764 - loss: 0.2862 - val_accuracy: 0.889

TypeError: cannot pickle 'module' object

In [ ]:
!nvidia-smi

## Version 3 - modificações

Divisão de Treinamento e Validação mais Completa:

A divisão atual de train[:80%] e train[80%:] pode ser melhorada usando tfds.split para garantir que os dados sejam divididos aleatoriamente.
Configuração de Autotuning para a Eficiência de Dados:

Ao criar os conjuntos de dados, adicionar AUTOTUNE para melhorar o desempenho do pipeline de dados.
Callbacks:

Usar callbacks, como ModelCheckpoint, EarlyStopping e ReduceLROnPlateau, pode ajudar a monitorar o desempenho do modelo durante o treinamento e fazer ajustes automaticamente.
Normalização:

A normalização das imagens pode ser feita com uma camada de Rescaling diretamente no modelo, tornando o pipeline mais limpo.
Comentários e Documentação:

Adicionar mais comentários para explicar as etapas pode ajudar na legibilidade do código.
Avaliação após o Treinamento:

Avaliar o modelo nos dados de teste (se disponíveis) após o treinamento para obter uma métrica de desempenho real.

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Definir parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 20
num_classes = 2
AUTOTUNE = tf.data.AUTOTUNE

# Carregar o dataset com divisão melhorada
(train_ds, validation_ds), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

# Função para pré-processar as imagens
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalização para [0, 1]
    image = tf.image.resize(image, (img_height, img_width))
    return image, label

# Aplicar pré-processamento e criar batches com AUTOTUNE
train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)
validation_ds = validation_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

# Aumentação de dados
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Construir o modelo CNN
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks para o treinamento
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
]

# Treinar o modelo
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
    callbacks=callbacks
)

# Salvar o modelo final
model.save('modelo_cnn_caes_gatos.h5')

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()


## otimização GPU

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Definir parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 20
num_classes = 2
AUTOTUNE = tf.data.AUTOTUNE

# Carregar o dataset com divisão melhorada
(train_ds, validation_ds), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

# Função para pré-processar as imagens
def preprocess_image(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # Normalização para [0, 1]
    image = tf.image.resize(image, (img_height, img_width))
    return image, label

# Aplicar pré-processamento e criar batches com AUTOTUNE
train_ds = train_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE).shuffle(1000).batch(batch_size).prefetch(AUTOTUNE)
validation_ds = validation_ds.map(preprocess_image, num_parallel_calls=AUTOTUNE).batch(batch_size).prefetch(AUTOTUNE)

# Aumentação de dados
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

# Construir o modelo CNN
model = models.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks para o treinamento
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/best_model.h5', save_best_only=True, monitor='val_loss', mode='min'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)
]

# Treinar o modelo
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=validation_ds,
    callbacks=callbacks
)

# Salvar o modelo final
model.save('/content/drive/MyDrive/modelo_cnn_caes_gatos.h5')

# Plotar o histórico de treinamento
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia do Modelo')
plt.xlabel('Época')
plt.ylabel('Acurácia')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Perda do Modelo')
plt.xlabel('Época')
plt.ylabel('Perda')
plt.legend()

plt.tight_layout()
plt.show()


## Testar Modelo CNN para Classificação de Cães e Gatos

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# Carregar o modelo treinado
model = tf.keras.models.load_model('modelo_cnn_caes_gatos.h5')

# Parâmetros
img_height, img_width = 150, 150

# Função para pré-processar a imagem
def preprocess_image(image):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (img_height, img_width))
    return image

# Carregar algumas imagens de teste
_, test_ds = tfds.load(
    'cats_vs_dogs',
    split=['train[:90%]', 'train[90%:]'],
    as_supervised=True,
)

# Selecionar algumas imagens aleatórias para teste
test_images = list(test_ds.shuffle(1000).take(10))

# Fazer previsões e mostrar resultados
plt.figure(figsize=(20, 20))
for i, (image, label) in enumerate(test_images):
    ax = plt.subplot(5, 2, i + 1)
    image = preprocess_image(image)
    image = tf.expand_dims(image, 0)  # Criar um batch de uma imagem
    prediction = model.predict(image)
    score = prediction[0][0]

    plt.imshow(tf.keras.preprocessing.image.array_to_img(image[0]))

    color = 'red'
    if (score > 0.5 and label == 1) or (score <= 0.5 and label == 0):
        color = 'green'

    plt.title(f"Previsão: {'Cachorro' if score > 0.5 else 'Gato'} ({score:.2f})\nReal: {'Cachorro' if label == 1 else 'Gato'}", color=color)
    plt.axis('off')

plt.tight_layout()
plt.show()

# Avaliar o modelo no conjunto de teste completo
test_ds = test_ds.map(lambda x, y: (preprocess_image(x), y)).batch(32)
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Acurácia no conjunto de teste: {test_accuracy:.2f}")

FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'modelo_cnn_caes_gatos.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Chat Interativo com Classificador de Imagens de Cães e Gatos

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import io

# Carregar o modelo treinado
model = tf.keras.models.load_model('modelo_cnn_caes_gatos.h5')

# Parâmetros
img_height, img_width = 150, 150

def preprocess_image(image):
    image = image.resize((img_height, img_width))
    image = np.array(image) / 127.5 - 1
    return np.expand_dims(image, 0)

def classify_image(image):
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    score = prediction[0][0]
    return "Cachorro" if score > 0.5 else "Gato", score

print("Bem-vindo ao Classificador de Cães e Gatos!")
print("Você pode fazer upload de uma imagem para classificação.")
print("Digite 'sair' a qualquer momento para encerrar o chat.")

while True:
    user_input = input("\nVocê: ")

    if user_input.lower() == 'sair':
        print("Assistente: Obrigado por usar o classificador. Até logo!")
        break

    elif user_input.lower() == 'classificar':
        print("Assistente: Por favor, faça o upload de uma imagem.")

        # Simular o upload de imagem
        print("(Simule o upload colando o caminho da imagem)")
        image_path = input("Caminho da imagem: ")

        try:
            with Image.open(image_path) as img:
                classification, confidence = classify_image(img)
                print(f"Assistente: A imagem parece ser de um {classification} com {confidence:.2%} de confiança.")
        except Exception as e:
            print(f"Assistente: Desculpe, ocorreu um erro ao processar a imagem: {str(e)}")

    else:
        print("Assistente: Se você quiser classificar uma imagem, digite 'classificar'. Se quiser sair, digite 'sair'.")

In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import tensorflow_datasets as tfds

# Parâmetros
img_height, img_width = 150, 150
batch_size = 32
epochs = 5  # Reduzido para um treinamento mais rápido

# Carregar e preparar o dataset
(train_ds, val_ds), metadata = tfds.load(
    'cats_vs_dogs',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
)

def preprocess_image(image, label):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (img_height, img_width))
    return image, label

train_ds = train_ds.map(preprocess_image).shuffle(1000).batch(batch_size)
val_ds = val_ds.map(preprocess_image).batch(batch_size)

# Construir e treinar o modelo
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

print("Treinando o modelo... Isso pode levar alguns minutos.")
model.fit(train_ds, validation_data=val_ds, epochs=epochs)
print("Treinamento concluído!")

def preprocess_single_image(image):
    image = image.resize((img_height, img_width))
    image = np.array(image) / 127.5 - 1
    return np.expand_dims(image, 0)

def classify_image(image):
    processed_image = preprocess_single_image(image)
    prediction = model.predict(processed_image)
    score = tf.nn.sigmoid(prediction[0][0]).numpy()
    return "Cachorro" if score > 0.5 else "Gato", score

print("\nBem-vindo ao Classificador de Cães e Gatos!")
print("Você pode fazer upload de uma imagem para classificação.")
print("Digite 'sair' a qualquer momento para encerrar o chat.")

while True:
    user_input = input("\nVocê: ")

    if user_input.lower() == 'sair':
        print("Assistente: Obrigado por usar o classificador. Até logo!")
        break

    elif user_input.lower() == 'classificar':
        print("Assistente: Por favor, faça o upload de uma imagem.")

        print("(Simule o upload colando o caminho da imagem)")
        image_path = input("Caminho da imagem: ")

        try:
            with Image.open(image_path) as img:
                classification, confidence = classify_image(img)
                print(f"Assistente: A imagem parece ser de um {classification} com {confidence:.2%} de confiança.")
        except Exception as e:
            print(f"Assistente: Desculpe, ocorreu um erro ao processar a imagem: {str(e)}")

    else:
        print("Assistente: Se você quiser classificar uma imagem, digite 'classificar'. Se quiser sair, digite 'sair'.")

Treinando o modelo... Isso pode levar alguns minutos.
Epoch 1/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 469s 795ms/step - accuracy: 0.5842 - loss: 0.6768 - val_accuracy: 0.7575 - val_loss: 0.5043
Epoch 2/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 434s 744ms/step - accuracy: 0.7462 - loss: 0.4985 - val_accuracy: 0.7797 - val_loss: 0.4341
Epoch 3/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 434s 741ms/step - accuracy: 0.7983 - loss: 0.4194 - val_accuracy: 0.7827 - val_loss: 0.4152
Epoch 4/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 426s 729ms/step - accuracy: 0.8305 - loss: 0.3644 - val_accuracy: 0.8177 - val_loss: 0.4183
Epoch 5/5
582/582 ━━━━━━━━━━━━━━━━━━━━ 429s 734ms/step - accuracy: 0.8623 - loss: 0.3043 - val_accuracy: 0.8203 - val_loss: 0.4277
Treinamento concluído!

Bem-vindo ao Classificador de Cães e Gatos!
Você pode fazer upload de uma imagem para classificação.
Digite 'sair' a qualquer momento para encerrar o chat.
